In [700]:
import pandas as pd
import numpy as np
from jieba import analyse
import os
import jieba
from sklearn import manifold
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# 自定义词典 & 停用词词典构建

In [948]:
##自定义携程词典
jieba.load_userdict(r"D:/Users/pjchang/Desktop/客户进线需求自动识别/stopwords-master/电话自定义词典&停用词词典/add_word.txt")

##停用词列表
g = open("D:/Users/pjchang/Desktop/客户进线需求自动识别/stopwords-master/电话自定义词典&停用词词典/cn_stopwords.txt",encoding = 'utf-8')
stopwordlist = []
for i in g.readlines():
    i = i.strip('\n')
    stopwordlist.append(i)
stopwordlist[-5:-1]

['麻烦', '稍', '到时候', '期间']

# 切词 关键词top10抽取

In [1052]:
#datasource = pd.read_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/IM端/输入/客人进线需求_im.xlsx")
#Cdatasource = pd.read_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/1024_IM_CALL_抽样300_tfidf_分词_员工话术.xlsx",sheet_name='电话')
Idatasource = pd.read_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/1024_IM_CALL_抽样300_tfidf_分词_员工话术.xlsx",sheet_name='IM')

datasource = Idatasource[['evntid','第一通客人进线内容','员工外呼酒店内容']]
datasource[['evntid','第一通客人进线内容','员工外呼酒店内容']]

,evntid,第一通客人进线内容,员工外呼酒店内容
0,271819062,"您好~,请问您是从哪里前往酒店的呢,酒店客服现在不在线呢，柠七是携程客服，帮您联系下酒店去核...",NaN
1,272082641,"中午好,收到,小管家帮您查看一下,您稍等3分钟,13362593999,您试试这个号码,看看...",NaN
2,271621531,"暖暖看一下图片,18867649192这个订单,暖暖帮您查询到这个订单不是您账户下的预定的，...",NaN
3,272393001,"稍等，我帮您查看一下,您拨打的是15516192749吗,您是需要咨询酒店什么呢,我可以为您...",NaN
4,272239344,"请问方便告知您是从哪个城市出发的~我来联系酒店咨询一下,为了更好的解决您的问题，东燕致电您订...",啊，您好，我这边是有个客人订了您酒店的房间叫高川，然后他这边想问一下酒店，他是通过旅行社预订...
...,...,...,...
295,272433007,"您好,兵兵来联系问一下,您是从哪里出发的,好的，您稍等10分钟,好的,酒店电话暂时没有接听，...",NaN
296,272492542,"您反馈的问题已收到，小威立刻为您查看，请您稍等一下哦~,为了节省您的时间，请问是否有其他事宜...","呃，行程码健康码都是绿码是吧，好的，然后其他就比如说有没有什么落地减之类的。,唉，您好，抱歉..."
297,271910888,"您反馈的问题已收到，小桃立即为您查看一下，请您稍等~,您好，请问您是已经到了酒店了对吗,小桃...","你好，来电，携程的，打扰到您啊，我们这边有个客人叫杨的，请问客人有办理好入住吗？,哦，已经出..."
298,271937457,"上午好😊，一琳来啦,一琳已明白您的诉求，我立马为您联系酒店协调处理，30分钟内在线回复您~,...","您好 携程旅行网 您好 我 这边 有 一个 ,您好 携程旅行网 ,哦 可以 的 是 吧 哦 ..."


In [1053]:
c = []
for i in range(len(datasource["第一通客人进线内容"].tolist())):
    c_stripblank = str(datasource["第一通客人进线内容"].tolist()[i]).replace(" ", "")
    c.append(c_stripblank)
e = datasource["员工外呼酒店内容"].tolist()

In [1054]:
##切词
cut_list_c = []
for i in c :
    cutwords = jieba.lcut(str(i))
    cutwords_c = [item for item in list(cutwords) if str(item) not in stopwordlist]
    cut_list_c.append(list(cutwords_c))

cut_list_e = []
for i in e :
    cutwords = jieba.lcut(str(i))
    cutwords_e = [item for item in list(cutwords) if str(item) not in stopwordlist]
    cut_list_e.append(list(cutwords_e))


In [1055]:
tfidf_dataset_c = []
for i in cut_list_c:
    a =(''.join(i))
    tfidf_dataset_c.append(a)

tfidf_dataset_e = []
for i in cut_list_e:
    a =(''.join(i))
    tfidf_dataset_e.append(a)


In [1058]:
##TFIDF 调用两次 改tfidf_dataset后缀c&e
tfidf = analyse.extract_tags
keywords_list = []
keywords_Wlist = []
for i in tfidf_dataset_e:
    Swords_list = []
    keywords = tfidf(str(i),topK=10,withWeight=True,allowPOS=("n","f","s","ns","vn","an","i","ng","nz","vg"))
    keywords_Wlist.append(keywords)
    for j in range(len(keywords)):
        Swords_list.append(keywords[j][0])
    keywords_list.append(Swords_list)
keywordsdf = pd.DataFrame(keywords_list)
keywords_Wdf = pd.DataFrame(keywords_Wlist)

#keywordsdf.columns = ['客关1','客关2','客关3','客关4','客关5','客关6','客关7','客关8','客关9','客关10']
#keywords_Wdf.columns = ['客关1w','客关2w','客关3w','客关4w','客关5w','客关6w','客关7w','客关8w','客关9w','客关10w']

keywordsdf.columns = ['员关1','员关2','员关3','员关4','员关5','员关6','员关7','员关8','员关9','员关10']
keywords_Wdf.columns = ['员关1w','员关2w','员关3w','员关4w','员关5w','员关6w','员关7w','员关8w','员关9w','员关10w']

In [213]:
"""
##textrank 调用两次 改tfidf_dataset后缀c&e
tr = analyse.textrank
keywords_list = []
keywords_Wlist = []
for i in tfidf_dataset_e:
    Swords_list = []
    keywords = tr(str(i),topK=10,withWeight=True,allowPOS=("n","f","s","ns","vn"))
    keywords_Wlist.append(keywords)
    for j in range(len(keywords)):
        Swords_list.append(keywords[j][0])
    keywords_list.append(Swords_list)
keywordsdf = pd.DataFrame(keywords_list)
keywords_Wdf = pd.DataFrame(keywords_Wlist)
##
keywordsdf.columns = ['客关1','客关2','客关3','客关4','客关5','客关6','客关7','客关8','客关9','客关10']
keywords_Wdf.columns = ['客关1w','客关2w','客关3w','客关4w','客关5w','客关6w','客关7w','客关8w','客关9w','客关10w']

#keywordsdf.columns = ['员关1','员关2','员关3','员关4','员关5','员关6','员关7','员关8','员关9','员关10']
#keywords_Wdf.columns = ['员关1w','员关2w','员关3w','员关4w','员关5w','员关6w','员关7w','员关8w','员关9w','员关10w']

In [1059]:
dfresult = pd.concat([dfresult,keywordsdf], axis=1)
#dfresult = pd.concat([dfresult, keywords_Wdf], axis=1)
dfresult

,evntid,第一通客人进线内容,员工外呼酒店内容,客关1,客关2,客关3,客关4,客关5,客关6,客关7,...,员关1,员关2,员关3,员关4,员关5,员关6,员关7,员关8,员关9,员关10
0,271819062,"您好~,请问您是从哪里前往酒店的呢,酒店客服现在不在线呢，柠七是携程客服，帮您联系下酒店去核...",NaN,业务经理,柠七,柠七问,心仪,关机,效劳,核实,...,None,None,None,None,None,None,None,None,None,None
1,272082641,"中午好,收到,小管家帮您查看一下,您稍等3分钟,13362593999,您试试这个号码,看看...",NaN,号码,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,271621531,"暖暖看一下图片,18867649192这个订单,暖暖帮您查询到这个订单不是您账户下的预定的，...",NaN,账户,家人,图片,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,272393001,"稍等，我帮您查看一下,您拨打的是15516192749吗,您是需要咨询酒店什么呢,我可以为您...",NaN,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,272239344,"请问方便告知您是从哪个城市出发的~我来联系酒店咨询一下,为了更好的解决您的问题，东燕致电您订...",啊，您好，我这边是有个客人订了您酒店的房间叫高川，然后他这边想问一下酒店，他是通过旅行社预订...,断线,城市,号码,高风险,行程,绿色,地区,...,订高川,旅行社,疫情,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,272433007,"您好,兵兵来联系问一下,您是从哪里出发的,好的，您稍等10分钟,好的,酒店电话暂时没有接听，...",NaN,入住率,销售,影响,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
296,272492542,"您反馈的问题已收到，小威立刻为您查看，请您稍等一下哦~,为了节省您的时间，请问是否有其他事宜...","呃，行程码健康码都是绿码是吧，好的，然后其他就比如说有没有什么落地减之类的。,唉，您好，抱歉...",出发地,地区,疫情,高风险,行程,节省,车站,...,疫情,行程,落地,政策,None,None,None,None,None,None
297,271910888,"您反馈的问题已收到，小桃立即为您查看一下，请您稍等~,您好，请问您是已经到了酒店了对吗,小桃...","你好，来电，携程的，打扰到您啊，我们这边有个客人叫杨的，请问客人有办理好入住吗？,哦，已经出...",小桃,办理,同事,None,None,None,None,...,打扰到,办理,数字,系统,None,None,None,None,None,None
298,271937457,"上午好😊，一琳来啦,一琳已明白您的诉求，我立马为您联系酒店协调处理，30分钟内在线回复您~,...","您好 携程旅行网 您好 我 这边 有 一个 ,您好 携程旅行网 ,哦 可以 的 是 吧 哦 ...",号码,核实,儿童,None,None,None,None,...,旅行网,家庭,儿童,早餐,None,None,None,None,None,None


# 客户&员工 共同高频率词

In [261]:
"""
trdfresult = pd.concat([datasource, keywordsdf], axis=1)
trdfresult = pd.concat([trdfresult, keywords_Wdf], axis=1)
trdfresult
"""

'\ntrdfresult = pd.concat([datasource, keywordsdf], axis=1)\ntrdfresult = pd.concat([trdfresult, keywords_Wdf], axis=1)\ntrdfresult\n'

In [215]:
#trdfresult.to_csv("D:/Users/pjchang/Desktop/textrank关键词_resultwith顺序.csv",encoding="utf_8_sig")

In [318]:
dfresult.to_csv("D:/Users/pjchang/Desktop/tfidf关键词_resultwith顺序1024withIM词库_IM.csv",encoding="utf_8_sig")

In [319]:
dfduplicate = dfresult[['客关1','客关2','客关3','客关4','客关5','客关6','客关7','客关8','客关9','客关10','员关1','员关2','员关3','员关4','员关5','员关6','员关7','员关8','员关9','员关10']]
extract_duplicates = []
for i in range(len(dfduplicate)):
    a = dfduplicate.iloc[i][dfduplicate.iloc[i].duplicated()][dfduplicate.iloc[i][dfduplicate.iloc[i].duplicated()].notnull()]
    extract_duplicates.append(a.tolist())
keywords_dropduplicate_df = pd.DataFrame(extract_duplicates)

In [320]:
keywords_dropduplicate_df

,0,1,2,3,4,5,6,7
0,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None
4,核酸,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
14601,None,None,None,None,None,None,None,None
14602,None,None,None,None,None,None,None,None
14603,接站,西站,车子,None,None,None,None,None
14604,None,None,None,None,None,None,None,None


In [321]:
dfwithduplicate = pd.concat([dfresult, keywords_dropduplicate_df], axis=1)
dfwithduplicate.to_csv("D:/Users/pjchang/Desktop/客户进线需求自动识别/tfidf关键词_resultwith顺序with重复词1024withIM词库_IM.csv",encoding="utf_8_sig")

# 【Tag-word】 词典读取

In [1060]:
datasourceCI = pd.read_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/关键词-600-11.02.xlsx",sheet_name="标签整理后1104")
datasourceCI

,标签,关键词,0,1,2,3,4,5,6,7,...,31,32,33,34,35,36,37,38,39,40
0,房型信息 - 房型信息,单间,标准间,双床,单人床,标间,双人,两张床,复式房,双床间,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,房型信息 - 售卖信息 - 担保,信用度,选项,银行信用,条款,预授权,担保金支付,支付担保金,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,房型信息 - 售卖信息 - 付款方式,外币,信用卡,微信,支付宝,在线付,前台现付,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,房型信息-宽带/无线,WIFI密码,宽带,无线密码,密码Wifi,密码wifi,密码无线网,密码无线,密码WiFi,密码WIFI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,房型信息-售卖信息-押金,押金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,基础信息 - 发票/水单,专票,发票,开票,开发票,纸质,普票,电子发票,开户行,税号抬头,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,基础信息 - 酒店电话,号码,手机,分机号,手机号码,停机,酒店电话,负责人电话,联系不上,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,基础信息-酒店地址,地址,找不到,车程分钟,车程距离,步行距离,分钟车程,距离车程,距离步行,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,基础信息-酒店邮箱,发邮件,邮件,邮箱,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,基础信息-酒店装修时间,装修时间,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1061]:
datasourceCI_around = pd.read_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/关键词-600-11.02.xlsx",sheet_name="关系模型")
dict_around = {}
for i in range(len(datasourceCI_around)):
    key1 = []
    key2 = []
    for j in datasourceCI_around.iloc[i,1:].tolist():
        if str(j) != 'nan':
            key1.append(str(j).split("+")[0])
            key2.append(str(j).split("+")[1])
    dict_around2 = {}
    for z in list(set(key1)):
        a = [q for q,x in enumerate(key1) if x == z]
        a_list = []
        for x in a:
            a_list.append(key2[x])
        dict_around2[str(z)] = a_list
    dict_around[str(datasourceCI_around.iloc[i,0])] = dict_around2
dict_around

{'房型信息 - 房型信息': {},
 '房型信息 - 售卖信息 - 担保': {'担保金': ['支付']},
 '房型信息 - 售卖信息 - 付款方式': {},
 '房型信息-宽带/无线': {'密码': ['Wifi', 'wifi', '无线网', '无线', 'WiFi', 'WIFI']},
 '房型信息-售卖信息-押金': {},
 '基础信息 - 发票/水单': {'税号': ['抬头']},
 '基础信息 - 酒店电话': {},
 '基础信息-酒店地址': {'步行': ['距离'], '车程': ['分钟', '距离']},
 '基础信息-酒店邮箱': {},
 '基础信息-酒店装修时间': {},
 '酒店政策-宠物政策-是否允许': {'宠物': ['携带']},
 '酒店政策-儿童政策': {},
 '酒店政策-入住政策-入住证件': {'身份证': ['电子'],
  '照片': ['身份证'],
  '派出所': ['附近'],
  '原件': ['身份证']},
 '酒店政策-入住政策-提前入住': {'提前入住': ['安排']},
 '酒店政策-入住政策-外宾': {'外宾': ['接待']},
 '酒店政策-停车场政策': {'停车场': ['免费', '地下', '公共', '收费']},
 '酒店政策-疫情政策': {'双码': ['核酸', '报告'],
  '核酸': ['48', '24', '72'],
  '落地': ['核酸'],
  '外地': ['核酸', '接待'],
  '政策': ['入住', '防疫'],
  '绿码': ['核酸'],
  '防疫': ['政策']},
 '酒店政策-早餐政策': {'收费': ['早餐'], '自助餐': ['早餐'], '早餐': ['含']},
 '其他信息-优惠券': {},
 '预订查询-酒店预订': {'团购': ['预约']},
 '设施信息 - 房型设施 - 洗衣机': {},
 '设施信息 - 酒店内部设施 - 接送服务-接送机 ': {},
 '设施信息-房型设施-电视机': {'投屏': ['电视']},
 '设施信息-房型设施-水池': {},
 '设施信息-房型设施-洗漱用品': {'拖鞋': ['一次性', '儿童'], '牙刷': 

In [1062]:
C_tag = set(datasourceCI['标签'].tolist())
C_tag = list(C_tag)

C_tag_l_all = []
for j in range(len(C_tag)):
    C_tag_lt = []
    for i in range(len(datasourceCI[datasourceCI['标签'] == C_tag[j]])):
        dfci = (datasourceCI[datasourceCI['标签'] == C_tag[j]].iloc[i])
        C_tag_lt += (list(dfci[1:].dropna()))
    
    C_tag_l = list(set(C_tag_lt))
    
    C_tag_l_all.append(C_tag_l)

In [1063]:
DFCI = pd.DataFrame(C_tag_l_all)
DFCI.insert(0,'Tag',C_tag)

In [69]:
#DFCI.to_csv("D:/Users/pjchang/Desktop/客户进线需求自动识别/标签词典.csv",encoding="utf_8_sig")

# 判断逻辑模块

In [1064]:
dict_ci = {}
for i in DFCI['Tag']:
    dict_ci [str(i)] = list(DFCI[DFCI['Tag'] == str(i)].iloc[0,1:].dropna())

In [1065]:
##判断命中词组模块
def contrast(l,diction,diction2):
    result = []
    Capture_Tag = []
    #单个词判断
    for key in diction.keys():
        lj = (set(l)&set(diction[key]))
        if len(list(lj)) != 0:
            result.append(key)
            Capture_Tag.append(list(lj))
    
    result_r = []
    Capture_list_r = []
    #关系词组判断
    for keyr in diction2.values():
        lk = set(l)&set(list(keyr))
        if len(list(lk)) != 0:
            ##被找到的关系词组key的list
            for keyr2 in list(lk):
                ##被找到的关系词组key的value list
                for keyr3 in keyr[keyr2]:
                    ##关系词组key在原文中前后3个词
                    indexa = list(set(l)).index(keyr2)
                    r_afterwords_j = []
                    for i in range(-3,4):
                        try:
                            r_afterwords_j.append(list(set(l))[indexa+i])
                        except:
                            r_afterwords_j.append("")
                    if keyr3 in r_afterwords_j:
                        key = list(dict_around.keys())[list(dict_around.values()).index(keyr)]
                        result_r.append(key)
                        Capture_list_r.append(keyr3)
                        print(keyr3,set(l),r_afterwords_j)
    Capture_Tag_F = list(Capture_Tag + list(set(Capture_list_r)))
    result1 = list(set(result + list(set(result_r))))
    return result1, Capture_Tag_F

def special_rule(tag_list):
    if "预订查询-酒店预订" in tag_list:
        tag_list = ["预订查询-酒店预订"]
    return tag_list

Capture_list = []
r_list = []
for i in range(len(dfresult)):
    l1 = list(dfresult.iloc[i,3:].dropna())
    a = contrast(l1,dict_ci,dict_around)
    b = special_rule(a[0])
    r_list.append(b)
    Capture_list.append(a[1])

免费 {'喜乐', '核酸', '环球', '园区', '姓名', '落地', '距离', '免费', '停车场', '市内', '旅行网', '地方', '时间段', '港号', '室外', '水手', '收费', '检测'} ['落地', '距离', '免费', '停车场', '市内', '旅行网', '地方']
政策 {'人员', '外来', '变化', '影响', '疫情', '地区', '政策', '监测', '部门', '防疫'} ['政策', '监测', '部门', '防疫', '', '', '']
防疫 {'人员', '外来', '变化', '影响', '疫情', '地区', '政策', '监测', '部门', '防疫'} ['影响', '疫情', '地区', '政策', '监测', '部门', '防疫']
防疫 {'号码', '核酸', '园园', '疫情', '行程', '服务', '政策', '防疫', '证明', '指挥部'} ['疫情', '行程', '服务', '政策', '防疫', '证明', '指挥部']
政策 {'号码', '核酸', '园园', '疫情', '行程', '服务', '政策', '防疫', '证明', '指挥部'} ['行程', '服务', '政策', '防疫', '证明', '指挥部', '']
核酸 {'换乘', '机票', '飞机', '核实', '核酸', '记录', '落地', '影响', '办理', '中心', '风景区', '地方', '行程', '部门'} ['核实', '核酸', '记录', '落地', '影响', '办理', '中心']
早餐 {'收费', '规定', '早餐'} ['收费', '规定', '早餐', '收费', '规定', '早餐', '']
无线 {'密码', '喜乐', '无线', '解决目前', '无人', '房号', '旅行网', '一切顺利', '别着急'} ['旅行网', '一切顺利', '别着急', '密码', '喜乐', '无线', '解决目前']
免费 {'指导', '核实', '陪伴', '定位', '免费', '信号', '手机号', '停车场', '精彩', '守候', '经历', '旅程'} ['免费', '信号', '手机号', '停车场', '精彩

In [1066]:
df_finalresult = pd.concat([datasource, pd.DataFrame(r_list), pd.DataFrame(Capture_list)], axis=1)
df_finalresult.to_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/模型判断结果11071.xlsx",sheet_name = 'CALL判断结果',encoding="utf_8_sig")

# N-Around

In [693]:
Tag_all = []
for i in range(len(C_tag_l_all)):
    Tag_all = Tag_all + C_tag_l_all[i]
Tag_all

['停车场',
 '停车位',
 '商务车',
 '航班号',
 '送机',
 '航站楼',
 '接机',
 '班机号',
 '调料',
 '午餐',
 '食材',
 '晚餐',
 '炉子',
 '餐具',
 '晚会',
 '行政酒廊',
 '炭火',
 '酒吧',
 '游泳池',
 '泳池',
 '游泳',
 '装修时间',
 '选项',
 '担保金',
 '预授权',
 '银行信用',
 '条款',
 '信用度',
 '浴帽',
 '梳子',
 '拖鞋',
 '牙刷',
 '用品',
 '低风险',
 '常态化',
 '旅居史',
 '双码',
 '核酸',
 '高风险',
 '阴性',
 '隔离区',
 '落地',
 '防控',
 '中风险',
 '黄码',
 '疫情',
 '健康码',
 '政策',
 '绿码',
 '防疫',
 '检测',
 'WIFI密码',
 '宽带',
 '无线密码',
 '两间房',
 '标准间',
 '续住',
 '标间',
 '主题房',
 '钟点',
 '报价',
 '星级建造',
 '无窗',
 '单人床',
 '亲子',
 '海景房',
 '房间数',
 '住几晚',
 '假日酒店',
 '观景',
 '销售部',
 '套装',
 '价位',
 '湖景',
 '星级',
 '预订',
 '房价格',
 '海景',
 '团购',
 '优惠价格',
 '民宿',
 '楼层',
 '洗衣服',
 '洗衣',
 '洗衣房',
 '酒店电话',
 '号码',
 '手机',
 '负责人电话',
 '停机',
 '联系不上',
 '分机号',
 '手机号码',
 '税号',
 '专票',
 '电子发票',
 '发票',
 '开票',
 '普票',
 '开发票',
 '纸质',
 '开户行',
 '入园',
 '园区',
 '套票',
 '套餐',
 '通票',
 '门票',
 '旁边',
 '周围',
 '核酸',
 '核酸点',
 '附近',
 '发邮件',
 '邮箱',
 '邮件',
 '行李',
 '行李寄存',
 '外币',
 '信用卡',
 '前台现付',
 '支付宝',
 '微信',
 '在线付',
 '猫猫',
 '宠物',
 '狗狗',
 '自助餐',
 '用餐',
 '早餐',
 '早餐券',
 '西式',
 '空房'

In [694]:
N_follow_dataset_c = []
for i in range(len(cut_list_c)):
    N_follow_dataset_c.extend(cut_list_c[i])

N_follow_dataset_e = []
for i in range(len(cut_list_e)):
    N_follow_dataset_e.extend(cut_list_e[i])

In [695]:
Tag_capture_N3_Cresult = []
for tag in Tag_all:
    Tag_capture_N3_Clist = []
    for i in [i for i,x in enumerate(N_follow_dataset_c) if x== tag]:
        try:
            Tag_capture_N3_Clist.append(N_follow_dataset_c[i-1])
        except:
            try:
                Tag_capture_N3_Clist.append(N_follow_dataset_c[i+1])
            except:
                try:
                    Tag_capture_N3_Clist.append(N_follow_dataset_c[i+2])
                except:
                    try:
                        Tag_capture_N3_Clist.append(N_follow_dataset_c[i+3])
                    except:
                        break
    for i in [i for i,x in enumerate(N_follow_dataset_e) if x== tag]:
        try:
            Tag_capture_N3_Clist.append(N_follow_dataset_e[i-1])
        except:
            try:
                Tag_capture_N3_Clist.append(N_follow_dataset_e[i+1])
            except:
                try:
                    Tag_capture_N3_Clist.append(N_follow_dataset_e[i+2])
                except:
                    try:
                        Tag_capture_N3_Clist.append(N_follow_dataset_e[i+3])
                    except:
                        break
    Tag_capture_N3_Clist.insert(0,tag)
    Tag_capture_N3_Cresult.append(Tag_capture_N3_Clist)

In [696]:
Tag_capture_N3_Cresult[0]

['停车场',
 '免费',
 '停',
 '岭',
 '免费',
 '入住',
 '确认',
 '告知',
 '收费',
 '地下',
 '确认',
 '号',
 '免费',
 '公共',
 '等待',
 '附近',
 '左右',
 '未有',
 '地方',
 '停车场',
 '地下',
 '正门口',
 '地下',
 '指引',
 '证明',
 '收费',
 '收费',
 '免费',
 '收费',
 '假日酒店',
 '公共',
 '号',
 '先',
 '以后',
 '一会',
 '地下',
 '地上',
 '提供',
 '公共',
 '一家',
 '久等',
 '地下',
 '地面',
 '地下',
 '地下',
 '地上',
 '旁边',
 '公共',
 '公共',
 '公共',
 '米',
 '两检',
 '三检',
 '地下',
 '地下',
 '告知',
 '室内',
 '免费',
 '私人',
 '提供',
 '免费',
 '提供',
 '专用',
 '免费',
 '地下',
 '免费',
 '收费',
 '附近',
 '等待',
 '宾馆酒店',
 '隔离',
 '分钟',
 '地下',
 '公共',
 '公共',
 '提供',
 '即可',
 '私人',
 '公共',
 '附近',
 '\xa0',
 '层',
 '免费',
 '地下',
 '免费',
 '公共',
 '地下',
 '一个',
 '大厦',
 '充电',
 '提供',
 '地下',
 '地下',
 '免费',
 '入住',
 '入住',
 '分离',
 '提供',
 '公共',
 '影城',
 '户外',
 '立即',
 '回复',
 '私人',
 '对面',
 '告知',
 '私人',
 '告知',
 '进入',
 '停车场',
 '告知',
 '号',
 '显示',
 '私人',
 '核实',
 '元',
 '地下',
 '地下',
 '号',
 '私人',
 '免费',
 '入住',
 '外面',
 '地面',
 '提供',
 '免费',
 '告知',
 '收费',
 '停',
 '公共',
 '门口',
 '告知',
 '营业',
 '度假区',
 '一个',
 '地下',
 '确认',
 '公共',
 '停',
 '收费',
 '北面',
 '等待',
 '对面',


In [697]:
Tag_P_dict = []
for i in range(len(Tag_capture_N3_Cresult)):
    Tag_p_dic = {}
    for target in list(set(Tag_capture_N3_Cresult[i])):
        Tag_P = (Tag_capture_N3_Cresult[i].count(target)/(len(Tag_capture_N3_Cresult[i])))
        Tag_p_dic[str(Tag_capture_N3_Cresult[i][0] + "+" + target)] = Tag_P
    Tag_P_dict.append(sorted(Tag_p_dic.items(),key = lambda d :d[1],reverse=True))

In [698]:
df1 = pd.DataFrame(Tag_P_dict[0])
for df_t in Tag_P_dict[1:]:
    df_t = pd.DataFrame(df_t)
    df1 = pd.concat([df1,df_t],axis=1,ignore_index=True)
df1

,0,1,2,3,4,5,6,7,8,9,...,360,361,362,363,364,365,366,367,368,369
0,停车场+免费,0.119839,停车位+,0.100000,商务车+那种,0.076923,航班号+,0.214286,送机+接机,0.090909,...,电子证件+告知,0.142857,身份证+电子,0.232908,派出所+附近,0.116883,原件+身份证,0.440000,临时身份证+办,0.15
1,停车场+,0.102719,停车位+免费,0.082353,商务车+有位,0.076923,航班号+提供,0.142857,送机+预约,0.060606,...,电子证件+照片,0.142857,身份证+,0.081112,派出所+,0.064935,原件+,0.080000,临时身份证+一个,0.15
2,停车场+地下,0.062437,停车位+现在,0.064706,商务车+安排,0.076923,航班号+方便,0.071429,送机+告知,0.045455,...,电子证件+是否,0.142857,身份证+提供,0.050985,派出所+告知,0.051948,原件+身份证件,0.066667,临时身份证+开具,0.15
3,停车场+公共,0.044310,停车位+告知,0.047059,商务车+接,0.076923,航班号+飞机,0.071429,送机+服务,0.045455,...,电子证件+遗失,0.071429,身份证+带,0.041715,派出所+先,0.038961,原件+入住,0.040000,临时身份证+身份证,0.10
4,停车场+告知,0.038268,停车位+门口,0.041176,商务车+包含,0.076923,航班号+告知,0.071429,送机+专车,0.045455,...,电子证件+电子证件,0.071429,身份证+入住,0.035921,派出所+入住,0.038961,原件+护照,0.040000,临时身份证+意思,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [699]:
df1.to_csv("D:/Users/pjchang/Desktop/客户进线需求自动识别/N3gram_result_IM1103.csv",encoding="utf_8_sig")